In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import sys
sys.path.append('../')

In [2]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from hydra import compose, initialize
from omegaconf import OmegaConf
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, ModelSummary
from torch.utils.data import DataLoader

from src.datasets import (CausalLMDataset, CausalLMPredictionDataset, MaskedLMDataset,
                          MaskedLMPredictionDataset, PaddingCollateFn)
from src.metrics import compute_metrics
from src.models import RNN, BERT4Rec, SASRec
from src.modules import SeqRec, SeqRecWithSampling
from src.postprocess import preds2recs
from src.preprocess import add_time_idx

## Config

In [3]:
# for SASRec
with initialize(version_base=None, config_path="../src/configs/"):
    config = compose(config_name="SASRec")

# # for BERT4Rec
# with initialize(version_base=None, config_path="../src/configs/"):
#     config = compose(config_name="BERT4Rec")

# # for GRU4Rec
# with initialize(version_base=None, config_path="../src/configs/"):
#     config = compose(config_name="RNN")

In [4]:
OmegaConf.set_struct(config, False)

config.dataset.max_length = 200

# # for training with negative sampling
# config.dataset.num_negatives = 1000

# # for original SASRec training with BCE loss and 1 negative example
# config.seqrec_module.loss = 'bce'
# config.dataset.num_negatives = 1
# config.dataset.full_negative_sampling = True

In [5]:
print(OmegaConf.to_yaml(config))

cuda_visible_devices: 0
data_path: ../data/ml-1m.txt
dataset:
  max_length: 200
  full_negative_sampling: false
dataloader:
  batch_size: 128
  test_batch_size: 256
  num_workers: 8
  validation_size: 10000
model: SASRec
model_params:
  maxlen: 200
  hidden_units: 64
  num_blocks: 2
  num_heads: 1
  dropout_rate: 0.1
seqrec_module:
  lr: 0.001
  predict_top_k: 10
  filter_seen: true
trainer_params:
  max_epochs: 100
patience: 10
sampled_metrics: false
top_k_metrics:
- 10
- 100



## Load data

In [6]:
data = pd.read_csv(config.data_path, sep=' ', header=None, names=['user_id', 'item_id'])
data = add_time_idx(data, sort=False)

# index 1 is used for masking value
if config.model == 'BERT4Rec':
    data.item_id += 1

print(data.shape)
data.head()

(999611, 4)


,user_id,item_id,time_idx,time_idx_reversed
0,1,1,0,78
1,1,2,1,77
2,1,3,2,76
3,1,4,3,75
4,1,5,4,74


In [7]:
train = data[data.time_idx_reversed >= 2]
validation = data[data.time_idx_reversed == 1]
validation_full = data[data.time_idx_reversed >= 1]
test = data[data.time_idx_reversed == 0]

## Dataloaders

In [8]:
validation_size = config.dataloader.validation_size
validation_users = validation_full.user_id.unique()
if validation_size and (validation_size < len(validation_users)):
    validation_users = np.random.choice(validation_users, size=validation_size, replace=False)

if config.model in ['SASRec', 'RNN']:
    train_dataset = CausalLMDataset(train, **config['dataset'])
    eval_dataset = CausalLMPredictionDataset(
        validation_full[validation_full.user_id.isin(validation_users)],
        max_length=config.dataset.max_length, validation_mode=True)
elif config.model == 'BERT4Rec':
    train_dataset = MaskedLMDataset(train, **config['dataset'])
    eval_dataset = MaskedLMPredictionDataset(
        validation_full[validation_full.user_id.isin(validation_users)],
        max_length=config.dataset.max_length, validation_mode=True)

train_loader = DataLoader(
    train_dataset, shuffle=True,
    collate_fn=PaddingCollateFn(),
    batch_size=config.dataloader.batch_size,
    num_workers=config.dataloader.num_workers)
eval_loader = DataLoader(
    eval_dataset, shuffle=False,
    collate_fn=PaddingCollateFn(),
    batch_size=config.dataloader.test_batch_size,
    num_workers=config.dataloader.num_workers)

In [9]:
batch = next(iter(train_loader))
print(batch['input_ids'].shape)

torch.Size([128, 200])


## Model

In [11]:
item_count = data.item_id.max()

if hasattr(config.dataset, 'num_negatives') and config.dataset.num_negatives:
    add_head = False
else:
    add_head = True

if config.model == 'SASRec':
    model = SASRec(item_num=item_count, add_head=add_head, **config.model_params)
if config.model == 'BERT4Rec':
    model = BERT4Rec(vocab_size=item_count + 1, add_head=add_head,
                     bert_config=config.model_params)
elif config.model == 'RNN':
    model = RNN(vocab_size=item_count + 1, add_head=add_head,
                rnn_config=config.model_params)

In [12]:
out = model(batch['input_ids'], batch['attention_mask'])
out.shape

torch.Size([128, 200, 3417])

## Train

In [13]:
if hasattr(config.dataset, 'num_negatives') and config.dataset.num_negatives:
    seqrec_module = SeqRecWithSampling(model, **config['seqrec_module'])
else:
    seqrec_module = SeqRec(model, **config['seqrec_module'])
    
early_stopping = EarlyStopping(monitor="val_ndcg", mode="max",
                               patience=config.patience, verbose=False)
model_summary = ModelSummary(max_depth=2)
checkpoint = ModelCheckpoint(save_top_k=1, monitor="val_ndcg",
                             mode="max", save_weights_only=True)
callbacks=[early_stopping, model_summary, checkpoint]

trainer = pl.Trainer(callbacks=callbacks, enable_checkpointing=True,
                     gpus=1, **config['trainer_params'])

trainer.fit(model=seqrec_module,
            train_dataloaders=train_loader,
            val_dataloaders=eval_loader)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name                       | Type       | Params
----------------------------------------------------------
0 | model                      | SASRec     | 282 K 
1 | model.item_emb             | Embedding  | 218 K 
2 | model.pos_emb              | Embedding  | 12.8 K
3 | model.emb_dropout          | Dropout    | 0     
4 | model.attention_layernorms | ModuleList | 256   
5 | model.attention_layers     | ModuleList | 33.3 K
6 | model.forward_layernorms   | ModuleList | 256   
7 | model.forward_layers       | ModuleList | 16.6 K
8 | model.last_layernorm       | LayerNorm  | 128   
----------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/home/antklen/anaconda3/envs/ptls/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (48) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [14]:
seqrec_module.load_state_dict(torch.load(checkpoint.best_model_path)['state_dict'])

<All keys matched successfully>

## Validation metrics

In [15]:
if config.model in ['SASRec', 'RNN']:
    predict_dataset = CausalLMPredictionDataset(train, max_length=config.dataset.max_length)
elif config.model  == 'BERT4Rec':
    predict_dataset = MaskedLMPredictionDataset(train, max_length=config.dataset.max_length)

predict_loader = DataLoader(
        predict_dataset, shuffle=False,
        collate_fn=PaddingCollateFn(),
        batch_size=config.dataloader.test_batch_size,
        num_workers=config.dataloader.num_workers)

seqrec_module.predict_top_k = max(config.top_k_metrics)
preds = trainer.predict(model=seqrec_module, dataloaders=predict_loader)

recs = preds2recs(preds)
print(recs.shape)
recs.head()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: 48it [00:00, ?it/s]

(604000, 3)


,user_id,item_id,prediction
0,1,737,7.598266
1,1,663,7.248270
2,1,102,7.237531
3,1,745,7.148354
4,1,1125,7.065181


In [16]:
for k in config.top_k_metrics:
    metrics_val = compute_metrics(validation, recs, k=k)
    print('k = ', k)
    print(metrics_val)

k =  10
{'ndcg@10': 0.18906012825101937, 'hit_rate@10': 0.33228476821192054, 'mrr@10': 0.14530563439503838}
k =  100
{'ndcg@100': 0.2657238370955717, 'hit_rate@100': 0.7066225165562914, 'mrr@100': 0.15960866987332128}


## Test metrics

In [17]:
if config.model in ['SASRec', 'RNN']:
    test_predict_dataset = CausalLMPredictionDataset(validation_full, max_length=config.dataset.max_length)
elif config.model  == 'BERT4Rec':
    test_predict_dataset = MaskedLMPredictionDataset(validation_full, max_length=config.dataset.max_length)
    
test_predict_loader = DataLoader(
        test_predict_dataset, shuffle=False,
        collate_fn=PaddingCollateFn(),
        batch_size=config.dataloader.test_batch_size,
        num_workers=config.dataloader.num_workers)

seqrec_module.predict_top_k = max(config.top_k_metrics)
preds_test = trainer.predict(model=seqrec_module, dataloaders=test_predict_loader)

recs_test = preds2recs(preds_test)
print(recs_test.shape)
recs_test.head()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: 48it [00:00, ?it/s]

(604000, 3)


,user_id,item_id,prediction
0,1,872,6.629054
1,1,667,6.363072
2,1,745,6.264328
3,1,679,6.156502
4,1,737,6.107877


In [18]:
for k in config.top_k_metrics:
    metrics_test = compute_metrics(test, recs_test, k=k)
    print('k = ', k)
    print(metrics_test)

k =  10
{'ndcg@10': 0.18250363127629773, 'hit_rate@10': 0.31473509933774835, 'mrr@10': 0.14229160096709764}
k =  100
{'ndcg@100': 0.25577861370312055, 'hit_rate@100': 0.6736754966887417, 'mrr@100': 0.15588159170341734}
